In [2]:
import numpy as np
import pandas as pd
from string import punctuation
import re
import datetime
import tqdm


pd.set_option('display.max_columns', 5000)

In [3]:
train = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone_Project\train.csv', dtype={'WindSpeed': 'object'})

In [4]:
train.head(22)

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
5,2017090700,20170907000118,away,75.06,24.00,1.01,0.32,0.18,308.34,95.01,2543494,Dee Ford,55,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-2,252,03/19/1991,Auburn,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
6,2017090700,20170907000118,away,74.11,16.64,1.11,0.83,0.02,357.23,322.59,2543637,Terrance Mitchell,39,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,5-11,190,05/17/1992,Oregon,CB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
7,2017090700,20170907000118,away,73.37,18.73,1.24,0.74,0.13,328.52,270.04,2543851,Phillip Gaines,23,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,193,04/04/1991,Rice,CB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
8,2017090700,20170907000118,away,56.63,26.90,0.26,1.86,0.28,344.70,55.31,2550257,Daniel Sorensen,49,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-2,208,03/05/1990,Brigham Young,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
9,2017090700,20170907000118,away,73.35,38.83,4.55,0.76,0.51,75.47,190.84,2552488,Marcus Peters,22,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB

In [5]:
def clean_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    txt = txt.replace('outside', 'outdoor')
    txt = txt.replace('outdor', 'outdoor')
    txt = txt.replace('outddors', 'outdoor')
    txt = txt.replace('outdoors', 'outdoor')
    txt = txt.replace('oudoor', 'outdoor')
    txt = txt.replace('indoors', 'indoor')
    txt = txt.replace('ourdoor', 'outdoor')
    txt = txt.replace('retractable', 'rtr.')
    return txt

In [6]:
train['StadiumType'] = train['StadiumType'].apply(clean_StadiumType)

In [7]:
def transform_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    if 'outdoor' in txt or 'open' in txt:
        return 1
    if 'indoor' in txt or 'closed' in txt:
        return 0
    
    return np.nan

In [8]:
train['StadiumType'] = train['StadiumType'].apply(transform_StadiumType)

In [9]:
#from https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/112681#latest-649087
Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
        'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
        'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
        'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 
        'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 

train['Turf'] = train['Turf'].map(Turf)
train['Turf'] = train['Turf'] == 'Natural'

In [10]:
map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
for abb in tqdm.tqdm(train['PossessionTeam'].unique()):
    map_abbr[abb] = abb

100%|███████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 32124.88it/s]


In [11]:
train['PossessionTeam'] = train['PossessionTeam'].map(map_abbr)
train['HomeTeamAbbr'] = train['HomeTeamAbbr'].map(map_abbr)
train['VisitorTeamAbbr'] = train['VisitorTeamAbbr'].map(map_abbr)

In [12]:
train['HomePossession'] = train['PossessionTeam'] == train['HomeTeamAbbr']

In [13]:
train['OffensePlayer'] = ((train['Team']=='home') & (train['HomePossession'])) | ((train['Team']=='away') & (train['HomePossession']==False))

In [14]:
train['Field_eq_Possession'] = train['FieldPosition'] == train['PossessionTeam']
train['HomeField'] = train['FieldPosition'] == train['HomeTeamAbbr']

In [15]:
train = pd.concat([train.drop(['OffenseFormation'], axis=1), pd.get_dummies(train['OffenseFormation'], prefix='Formation')], axis=1)
dummy_col = train.columns

In [16]:
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

In [17]:
train['GameClock'] = train['GameClock'].apply(strtoseconds)

In [18]:
train['PlayerHeight'] = train['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

In [19]:
train['PlayerBMI'] = 703*(train['PlayerWeight']/(train['PlayerHeight'])**2)

In [20]:
train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

In [21]:
train['TimeDelta'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)

In [22]:
train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

In [23]:
seconds_in_year = 60*60*24*365.25
train['PlayerAge'] = train.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)

In [24]:
train['WindSpeed'] = train['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)

In [25]:
#let's replace the ones that has x-y by (x+y)/2
# and also the ones with x gusts up to y
train['WindSpeed'] = train['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
train['WindSpeed'] = train['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)

In [26]:
def str_to_float(txt):
    try:
        return float(txt)
    except:
        return -1

In [27]:
train['WindSpeed'] = train['WindSpeed'].apply(str_to_float)

In [28]:
def clean_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = txt.replace('from', '')
    txt = txt.replace(' ', '')
    txt = txt.replace('north', 'n')
    txt = txt.replace('south', 's')
    txt = txt.replace('west', 'w')
    txt = txt.replace('east', 'e')
    return txt

In [29]:
train['WindDirection'] = train['WindDirection'].apply(clean_WindDirection)

In [30]:
def transform_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    
    if txt=='n':
        return 0
    if txt=='nne' or txt=='nen':
        return 1/8
    if txt=='ne':
        return 2/8
    if txt=='ene' or txt=='nee':
        return 3/8
    if txt=='e':
        return 4/8
    if txt=='ese' or txt=='see':
        return 5/8
    if txt=='se':
        return 6/8
    if txt=='ses' or txt=='sse':
        return 7/8
    if txt=='s':
        return 8/8
    if txt=='ssw' or txt=='sws':
        return 9/8
    if txt=='sw':
        return 10/8
    if txt=='sww' or txt=='wsw':
        return 11/8
    if txt=='w':
        return 12/8
    if txt=='wnw' or txt=='nww':
        return 13/8
    if txt=='nw':
        return 14/8
    if txt=='nwn' or txt=='nnw':
        return 15/8
    return np.nan

In [31]:
train['WindDirection'] = train['WindDirection'].apply(transform_WindDirection)

In [32]:
train['PlayDirection'] = train['PlayDirection'].apply(lambda x: x.strip() == 'right')

In [33]:
train['GameWeather'] = train['GameWeather'].str.lower()
indoor = "indoor"
train['GameWeather'] = train['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)

In [34]:
def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

In [35]:
train['GameWeather'] = train['GameWeather'].apply(map_weather)

In [36]:
train['IsRusher'] = train['NflId'] == train['NflIdRusher']

In [37]:
train.drop(['NflId', 'NflIdRusher'], axis=1, inplace=True)

In [38]:
train['X'] = train.apply(lambda row: row['X'] if row['PlayDirection'] else 120-row['X'], axis=1)

In [39]:
train['Y'] = train.apply(lambda row: row['Y'] if row['PlayDirection'] else 53.33-row['Y'], axis=1)

In [40]:
#from https://www.kaggle.com/scirpus/hybrid-gp-and-nn
def new_orientation(angle, play_direction):
    if play_direction == 0:
        new_angle = 360.0 - angle
        if new_angle == 360.0:
            new_angle = 0.0
        return new_angle
    else:
        return angle
    
train['Orientation'] = train.apply(lambda row: new_orientation(row['Orientation'], row['PlayDirection']), axis=1)
train['Dir'] = train.apply(lambda row: new_orientation(row['Dir'], row['PlayDirection']), axis=1)

In [91]:
train['YardsLeft'] = train.apply(lambda row: 100-row['YardLine'] if row['Field_eq_Possession'] else row['YardLine'], axis=1)

In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682154 entries, 0 to 682153
Data columns (total 63 columns):
GameId                    682154 non-null int64
PlayId                    682154 non-null int64
Team                      682154 non-null object
X                         682154 non-null float64
Y                         682154 non-null float64
S                         682154 non-null float64
A                         682154 non-null float64
Dis                       682154 non-null float64
Orientation               682131 non-null float64
Dir                       682126 non-null float64
DisplayName               682154 non-null object
JerseyNumber              682154 non-null int64
Season                    682154 non-null int64
YardLine                  682154 non-null int64
Quarter                   682154 non-null int64
GameClock                 682154 non-null float64
PossessionTeam            682154 non-null object
Down                      682154 non-null int64
Distan

In [43]:
train[pd.isna(train.StadiumType)].Stadium.unique()

array(['NRG Stadium', 'StubHub Center', 'MetLife Stadium',
       'University of Phoenix Stadium', 'NRG', 'Heinz Field',
       'State Farm Stadium', 'TIAA Bank Field', 'Mercedes-Benz Superdome',
       'Dignity Health Sports Park', 'Mercedes-Benz Stadium'],
      dtype=object)

In [44]:
indoor = ['NRG Stadium', 'University of Phoenix Stadium', 'NRG', 'State Farm Stadium', 'Mercedes-Benz Superdome', 'Mercedes-Benz Stadium']
outdoor = ['StubHub Center', 'MetLife Stadium', 'Heinz Field', 'TIAA Bank Field', 'Dignity Health Sports Park']

mask = np.in1d(train.Stadium, indoor)
train['StadiumType'][mask] = 1

mask = np.in1d(train.Stadium, outdoor)
train['StadiumType'][mask] = 0

C:\Users\Evan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Evan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
train.drop(['FieldPosition'], axis=1, inplace=True)

In [46]:
values = {'Temperature':65, 'Humidity':50, 'WindSpeed':0, 'WindDirection':0}
train.fillna(value=values, inplace=True)

In [47]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682154 entries, 0 to 682153
Data columns (total 62 columns):
GameId                    682154 non-null int64
PlayId                    682154 non-null int64
Team                      682154 non-null object
X                         682154 non-null float64
Y                         682154 non-null float64
S                         682154 non-null float64
A                         682154 non-null float64
Dis                       682154 non-null float64
Orientation               682131 non-null float64
Dir                       682126 non-null float64
DisplayName               682154 non-null object
JerseyNumber              682154 non-null int64
Season                    682154 non-null int64
YardLine                  682154 non-null int64
Quarter                   682154 non-null int64
GameClock                 682154 non-null float64
PossessionTeam            682154 non-null object
Down                      682154 non-null int64
Distan

In [72]:
train.Position.unique()

array(['SS', 'DE', 'ILB', 'FS', 'CB', 'DT', 'WR', 'TE', 'T', 'QB', 'RB',
       'G', 'C', 'OLB', 'NT', 'FB', 'MLB', 'LB', 'OT', 'S', 'OG', 'HB',
       'DB', 'DL', 'SAF'], dtype=object)

In [73]:
de = train['PlayerWeight'].where(train['Position']=='DE')
de_wt = de.groupby(de.index // 22).sum()

In [74]:
dt = train['PlayerWeight'].where(train['Position']=='DT')
dt_wt = dt.groupby(dt.index // 22).sum()

In [76]:
nt = train['PlayerWeight'].where(train['Position']=='NT')
nt_wt = nt.groupby(nt.index // 22).sum()

In [77]:
dl = train['PlayerWeight'].where(train['Position']=='DL')
dl_wt = nt.groupby(dl.index // 22).sum()

In [79]:
dline_wt = de_wt + dt_wt + nt_wt + dl_wt
dline_wt

0        1118.0
1        1118.0
2        1118.0
3        1796.0
4         935.0
5        1460.0
6        1747.0
7        1748.0
8         935.0
9         935.0
10        935.0
11       1459.0
12       1130.0
13       1748.0
14       1748.0
15        900.0
16       1748.0
17       1748.0
18       1748.0
19       1418.0
20       1418.0
21       1438.0
22       1767.0
23       1796.0
24       1250.0
25        930.0
26        930.0
27       1440.0
28        935.0
29        900.0
          ...  
30977     950.0
30978     895.0
30979    1207.0
30980     895.0
30981     585.0
30982     585.0
30983    1170.0
30984     895.0
30985     895.0
30986    1192.0
30987     910.0
30988     895.0
30989    1207.0
30990     895.0
30991    1207.0
30992     867.0
30993     912.0
30994    1222.0
30995    1207.0
30996     895.0
30997     623.0
30998    1222.0
30999    1222.0
31000     910.0
31001    1207.0
31002    1207.0
31003     897.0
31004     897.0
31005    1222.0
31006     912.0
Name: PlayerWeight, Leng

In [80]:
t = train['PlayerWeight'].where(train['Position']=='T')
t_wt = t.groupby(t.index // 22).sum()

In [83]:
og = train['PlayerWeight'].where(train['Position']=='OG')
og_wt = og.groupby(og.index // 22).sum()

In [84]:
ot = train['PlayerWeight'].where(train['Position']=='OT')
ot_wt = ot.groupby(ot.index // 22).sum()

In [85]:
c = train['PlayerWeight'].where(train['Position']=='C')
c_wt = c.groupby(c.index // 22).sum()

In [86]:
g = train['PlayerWeight'].where(train['Position']=='G')
g_wt = g.groupby(c.index // 22).sum()

In [88]:
oline_wt = t_wt + og_wt + ot_wt + c_wt + g_wt
oline_wt

0        1578.0
1        1578.0
2        1578.0
3        1893.0
4        1581.0
5        1578.0
6        1578.0
7        1893.0
8        1581.0
9        1581.0
10       1581.0
11       1578.0
12       1578.0
13       1578.0
14       1578.0
15       1581.0
16       1578.0
17       1578.0
18       1578.0
19       1578.0
20       1578.0
21       1578.0
22       1578.0
23       1893.0
24       1581.0
25       1581.0
26       1581.0
27       1578.0
28       1581.0
29       1581.0
          ...  
30977    1555.0
30978    1590.0
30979    1590.0
30980    1590.0
30981    1590.0
30982    1590.0
30983    1590.0
30984    1590.0
30985    1590.0
30986    1590.0
30987    1590.0
30988    1590.0
30989    1590.0
30990    1590.0
30991    1590.0
30992    1590.0
30993    1590.0
30994    1590.0
30995    1590.0
30996    1590.0
30997    1555.0
30998    1595.0
30999    1595.0
31000    1595.0
31001    1595.0
31002    1595.0
31003    1595.0
31004    1595.0
31005    1595.0
31006    1595.0
Name: PlayerWeight, Leng

In [90]:
OlineDlineRatio = oline_wt / dline_wt
OlineDlineRatio

0        1.411449
1        1.411449
2        1.411449
3        1.054009
4        1.690909
5        1.080822
6        0.903263
7        1.082952
8        1.690909
9        1.690909
10       1.690909
11       1.081563
12       1.396460
13       0.902746
14       0.902746
15       1.756667
16       0.902746
17       0.902746
18       0.902746
19       1.112835
20       1.112835
21       1.097357
22       0.893039
23       1.054009
24       1.264800
25       1.700000
26       1.700000
27       1.095833
28       1.690909
29       1.756667
           ...   
30977    1.636842
30978    1.776536
30979    1.317316
30980    1.776536
30981    2.717949
30982    2.717949
30983    1.358974
30984    1.776536
30985    1.776536
30986    1.333893
30987    1.747253
30988    1.776536
30989    1.317316
30990    1.776536
30991    1.317316
30992    1.833910
30993    1.743421
30994    1.301146
30995    1.317316
30996    1.776536
30997    2.495987
30998    1.305237
30999    1.305237
31000    1.752747
31001    1

In [92]:
train.to_csv(r'C:\Users\Evan\Desktop\Capstone_Project\clean_train.csv')

In [81]:
train.Position.unique()

array(['SS', 'DE', 'ILB', 'FS', 'CB', 'DT', 'WR', 'TE', 'T', 'QB', 'RB',
       'G', 'C', 'OLB', 'NT', 'FB', 'MLB', 'LB', 'OT', 'S', 'OG', 'HB',
       'DB', 'DL', 'SAF'], dtype=object)